In [1]:
import pandas as pd
import numpy as np

from pandas.tseries.offsets import MonthEnd
from datetime import datetime

In [2]:
portfolio_res = pd.read_csv('portfolio_res.csv', index_col='report_dt')
agg_res = pd.read_csv('agg_res.csv', index_col='Unnamed: 0')

In [3]:
del portfolio_res['Unnamed: 0']

## Необходимые функции

Из общего портфеля (portfolio_res) делаем агрегированный (agg_res)  

In [4]:
del_cols = ['month_maturity', 'bucketed_balance_nm', 'bucketed_balance', 'weight_rate', 'gen_name', 'renewal_balance_next_month']

In [5]:
portfolio_res = portfolio_res.drop(columns=del_cols)

In [42]:
portfolio_res.columns

Index(['segment', 'replenishable_flg', 'subtraction_flg',
       'target_maturity_days', 'open_month', 'close_month', 'balance',
       'renewal_cnt', 'operations_in_month', 'early_withdrawal_in_month',
       'report_month', 'ew_share'],
      dtype='object')

In [7]:
_REPORT_DT_COLUMN = 'report_dt'

In [8]:
GROUP_AGG_COLS = ['segment', 'replenishable_flg', 'subtraction_flg', 'target_maturity_days']

In [9]:
portfolio_res.index = pd.to_datetime(portfolio_res.index)

In [10]:
pd.to_datetime(portfolio_res.index.unique())

DatetimeIndex(['2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30'], dtype='datetime64[ns]', name='report_dt', freq=None)

In [11]:
portfolio_res.head(3)

,segment,replenishable_flg,subtraction_flg,target_maturity_days,open_month,close_month,balance,renewal_cnt,operations_in_month,early_withdrawal_in_month
report_dt,,,,,,,,,,
2023-08-31,mass,0,0,90,2023-08,2023-11,5.587933e+10,0.0,0.0,NaN
2023-08-31,mass,0,0,180,2023-03,2023-09,3.102769e+10,0.0,0.0,-2.957130e+08
2023-08-31,mass,0,0,180,2023-03,2023-09,3.052281e+10,0.0,0.0,-1.910815e+08


In [12]:
def _agg_contract_close_fact(port, group_cols):
    res = pd.DataFrame()
    for date in pd.to_datetime(portfolio_res.index.unique())[1:]:
        previous_dt = date + MonthEnd(-1)
        previos_port_dt = port.loc[previous_dt]
        close_port = previos_port_dt[previos_port_dt.close_month == date.strftime('%Y-%m')]

        renewal = port.loc[date]
        renewal = renewal[(renewal['open_month'].values==pd.Series(renewal.index).astype(str).apply(lambda x: x[:7]).values)&(renewal['renewal_cnt']>=1)]

        
        renewal = renewal.rename(columns={'balance': 'renewal_balance_next_month'})
        renewal = renewal[group_cols+['renewal_balance_next_month']]


        agg_close = close_port.groupby(['close_month'] + group_cols)[['balance']].sum().reset_index()
        agg_renewal = renewal.groupby(group_cols)[['renewal_balance_next_month']].sum().reset_index()
        
        agg_close = agg_close.merge(agg_renewal, on=group_cols, how='left')
        
        res = res.append(agg_close)
    res['contract_close'] = (res['balance'] - res['renewal_balance_next_month'])*-1
    res = res.rename(columns = {'close_month': _REPORT_DT_COLUMN})
    res[_REPORT_DT_COLUMN] = pd.to_datetime(res[_REPORT_DT_COLUMN]) + MonthEnd(0)
    return res[[_REPORT_DT_COLUMN] + group_cols + ['contract_close']]

In [13]:
_agg_contract_close_fact(portfolio_res, GROUP_AGG_COLS)#.groupby('report_dt').sum()

,report_dt,segment,replenishable_flg,subtraction_flg,target_maturity_days,contract_close
0,2023-09-30,mass,0,0,90,-1.132872e+10
1,2023-09-30,mass,0,0,180,-9.581471e+10
2,2023-09-30,mass,0,0,365,-7.249158e+10
3,2023-09-30,mass,0,0,548,NaN
4,2023-09-30,mass,0,0,730,-5.580819e+06
...,...,...,...,...,...,...
38,2023-11-30,vip,1,1,90,-3.972814e+11
39,2023-11-30,vip,1,1,180,-4.670417e+09
40,2023-11-30,vip,1,1,365,-3.317851e+08
41,2023-11-30,vip,1,1,548,NaN


In [14]:
portfolio_res

,segment,replenishable_flg,subtraction_flg,target_maturity_days,open_month,close_month,balance,renewal_cnt,operations_in_month,early_withdrawal_in_month
report_dt,,,,,,,,,,
2023-08-31,mass,0,0,90,2023-08,2023-11,5.587933e+10,0.0,0.0,NaN
2023-08-31,mass,0,0,180,2023-03,2023-09,3.102769e+10,0.0,0.0,-2.957130e+08
2023-08-31,mass,0,0,180,2023-03,2023-09,3.052281e+10,0.0,0.0,-1.910815e+08
2023-08-31,mass,0,0,365,2022-09,2023-09,2.480401e+10,0.0,0.0,-1.934412e+08
2023-08-31,mass,0,0,180,2023-06,2023-12,1.788060e+10,0.0,0.0,-4.843801e+08
...,...,...,...,...,...,...,...,...,...,...
2023-11-30,vip,1,1,730,2022-10,2024-10,1.011292e+03,0.0,0.0,0.000000e+00
2023-11-30,vip,1,1,365,2021-06,2024-06,1.000385e+03,2.0,0.0,0.000000e+00
2023-11-30,vip,1,1,365,2015-02,2024-02,1.740044e+00,2.0,0.0,0.000000e+00


In [15]:
def _agg_renewal(port, group_cols):
    port['report_month'] = pd.Series(port.index).astype(str).apply(lambda x: x[:7]).values
    renewal = port.query('(report_month == open_month) and (renewal_cnt >= 1) ')

    renewal = renewal.groupby([_REPORT_DT_COLUMN] + group_cols)['balance'].sum().reset_index()
    renewal = renewal.rename(columns = {'balance':'renewal'})
    return renewal

In [16]:
_agg_renewal(portfolio_res, GROUP_AGG_COLS).groupby('report_dt').sum()

,replenishable_flg,subtraction_flg,target_maturity_days,renewal
report_dt,,,,
2023-09-30,17,17,12848,6.550854e+10
2023-10-31,16,16,12300,4.261724e+10
2023-11-30,16,16,13303,2.644586e+11


In [17]:
agg_res.groupby('report_dt').sum()['renewal']

report_dt
2023-09-30    6.550854e+10
2023-10-31    4.261724e+10
2023-11-30    2.644586e+11
Name: renewal, dtype: float64

In [18]:
def _agg_newbusiness(port, group_cols):
    port['report_month'] = pd.Series(port.index).astype(str).apply(lambda x: x[:7]).values
    newbiz = port.query('(report_month == open_month) and (renewal_cnt < 1) ')

    newbiz = newbiz.groupby([_REPORT_DT_COLUMN, *group_cols])['balance'].sum().reset_index()
    newbiz = newbiz.rename(columns = {'balance':'newbusiness'})
    min_date = newbiz[_REPORT_DT_COLUMN].min()
    return newbiz[newbiz[_REPORT_DT_COLUMN]>min_date]

In [19]:
_agg_newbusiness(portfolio_res, GROUP_AGG_COLS).groupby('report_dt').sum()

,replenishable_flg,subtraction_flg,target_maturity_days,newbusiness
report_dt,,,,
2023-09-30,24,18,21056,7.635291e+11
2023-10-31,24,18,21056,5.077073e+11
2023-11-30,24,18,21056,6.542739e+11


In [20]:
agg_res.groupby('report_dt').sum()['newbusiness']

report_dt
2023-09-30    7.635291e+11
2023-10-31    5.077073e+11
2023-11-30    6.542739e+11
Name: newbusiness, dtype: float64

In [21]:
def _agg_early_withdrawal(port, group_cols):
    ew = port.groupby([_REPORT_DT_COLUMN, *group_cols])[['operations_in_month', 'early_withdrawal_in_month']].sum().reset_index()
    ew['early_withdrawal'] = np.where(
        (ew.replenishable_flg > 0) | (ew.subtraction_flg > 0),
        0,
        ew.early_withdrawal_in_month
    )
        ew['operations'] = np.where(
        (ew.replenishable_flg > 0) | (ew.subtraction_flg > 0),
        ew.operations_in_month,
        0
    )
    min_date = ew[_REPORT_DT_COLUMN].min()
    ew = ew[ew[_REPORT_DT_COLUMN]>min_date]
    return ew[[_REPORT_DT_COLUMN, *group_cols, 'early_withdrawal', 'operations']]

In [22]:
_agg_early_withdrawal(portfolio_res, GROUP_AGG_COLS).groupby('report_dt').sum()

,replenishable_flg,subtraction_flg,target_maturity_days,early_withdrawal,operations
report_dt,,,,,
2023-09-30,31,26,31643,-4.511836e+10,-2.186902e+10
2023-10-31,30,25,30913,-2.410642e+10,-1.994758e+10
2023-11-30,30,25,30913,-2.111107e+10,-4.711145e+09


In [23]:
agg_res.groupby('report_dt').sum()[['early_withdrawal', 'operations']]

,early_withdrawal,operations
report_dt,,
2023-09-30,-4.511836e+10,-2.186902e+10
2023-10-31,-2.410642e+10,-1.994758e+10
2023-11-30,-2.111107e+10,-4.711145e+09


In [24]:
def _agg_interests(port, group_cols):
    # balance_lag1 - рассчитываем как сдивнутый баланс на один временной срез. далее соединяем по ключам group_cols
    old_port = port[port.balance_lag1 > 0]
    #SER_d_cl - это сумма operations_in_month и early_withdrawal_in_month
    old_port['interests'] = old_port['balance'] - old_port['balance_lag1'].astype(float) - old_port['SER_d_cl'] 
    interests = old_port.groupby([_REPORT_DT_COLUMN, *group_cols])['interests'].sum().reset_index()
    return interests

In [25]:
def _agg_balance(port, group_cols):
    bal = port.groupby([_REPORT_DT_COLUMN, *group_cols])['balance'].sum().reset_index()
    
    min_date = bal[_REPORT_DT_COLUMN].min()
    bal = bal[bal[_REPORT_DT_COLUMN]>min_date]
    return bal

In [26]:
_agg_balance(portfolio_res, GROUP_AGG_COLS).groupby('report_dt').sum()

,replenishable_flg,subtraction_flg,target_maturity_days,balance
report_dt,,,,
2023-09-30,31,26,31643,3.634163e+12
2023-10-31,30,25,30913,3.835058e+12
2023-11-30,30,25,30913,3.721102e+12


In [27]:
agg_res.groupby('report_dt').sum()['balance']

report_dt
2023-09-30    3.634163e+12
2023-10-31    3.835058e+12
2023-11-30    3.721102e+12
Name: balance, dtype: float64

In [28]:
def _aggregated_data(port, group_cols):
    contract_close = _agg_contract_close_fact(port, group_cols)
    renewal = _agg_renewal(port, group_cols)
    newbiz = _agg_newbusiness(port, group_cols)
    ew_oper = _agg_early_withdrawal(port, group_cols)
    # требуе небольшой доработки
    #interests = _agg_interests(port, group_cols)
    balance = _agg_balance(port, group_cols)
    res = reduce(lambda x,y: x.merge(y, on = [_REPORT_DT_COLUMN, *group_cols], how = 'outer'), [balance, newbiz, contract_close, ew_oper, renewal]) # interests добавить
    return res.fillna(0)

In [29]:
from functools import reduce

In [30]:
agg_data = _aggregated_data(portfolio_res, GROUP_AGG_COLS)

In [31]:
agg_data.groupby('report_dt').sum()

,replenishable_flg,subtraction_flg,target_maturity_days,balance,newbusiness,contract_close,early_withdrawal,operations,renewal
report_dt,,,,,,,,,
2023-09-30,32,27,32373,3.634163e+12,7.635291e+11,-3.777065e+11,-4.511836e+10,-2.186902e+10,6.550854e+10
2023-10-31,31,26,31643,3.835058e+12,5.077073e+11,-2.743720e+11,-2.410642e+10,-1.994758e+10,4.261724e+10
2023-11-30,30,25,30913,3.721102e+12,6.542739e+11,-7.513916e+11,-2.111107e+10,-4.711145e+09,2.644586e+11


In [32]:
agg_res.groupby('report_dt').sum()

,replenishable_flg,subtraction_flg,month_maturity,target_maturity_days,balance_start,balance_gain,balance,newbusiness,contract_close,early_withdrawal,operations,interests,renewal,universal_weight_id
report_dt,,,,,,,,,,,,,,
2023-09-30,32,27,1065,32373,3.295017e+12,3.391453e+11,3.634163e+12,7.635291e+11,-3.777930e+11,-4.511836e+10,-2.186902e+10,2.039656e+10,6.550854e+10,304
2023-10-31,31,26,1041,31643,3.634163e+12,2.008951e+11,3.835058e+12,5.077073e+11,-2.880336e+11,-2.410642e+10,-1.994758e+10,2.527537e+10,4.261724e+10,294
2023-11-30,30,25,1017,30913,3.835058e+12,-1.139559e+11,3.721102e+12,6.542739e+11,-7.642714e+11,-2.111107e+10,-4.711145e+09,2.186379e+10,2.644586e+11,284


## Логика расчета досрочного доли отзыва

Рассчитываем долю от текущего баланса по следующей логике:  
доля = досрочный_отзыв / (сумма_баланс + сумма_досрочный_отзыв)  

Сумма рассчитывается на основе группировки

In [33]:
group_cols_share = ['report_dt', 'segment', 'replenishable_flg', 'subtraction_flg', 'target_maturity_days']

In [34]:
agg_res_sh = agg_res.groupby(group_cols_share).sum()

In [35]:
agg_res_sh['sum_bal'] = agg_res_sh['balance'] + agg_res_sh['early_withdrawal'].abs()

In [36]:
agg_res = agg_res.merge(agg_res_sh['sum_bal'], on=group_cols_share, how='left')

In [37]:
agg_res['ew_share'] = agg_res['early_withdrawal'].abs() / agg_res['sum_bal'] 

In [40]:
portfolio_res['ew_share'] = portfolio_res['early_withdrawal_in_month'].abs() / portfolio_res['balance'] 

In [41]:
portfolio_res.to_csv('portfolio_res_ew.csv')